In [280]:
## Import modules

from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
from backtesting.test import GOOG
import MetaTrader5 as mt
import pandas as pd
import pandas_ta as ta
from datetime import datetime
import matplotlib.pyplot as plt
from math import sqrt
import seaborn as sns
import numpy as np

In [279]:
## Sync to MetaTrader5

mt.initialize()

True

In [232]:
## Extract data we want to work on

df = pd.DataFrame(mt.copy_rates_range('[NQ100]', 
                                            mt.TIMEFRAME_M5, 
                                            datetime(2023, 5, 1), 
                                            datetime.now()))

df['time'] = pd.to_datetime(df['time'], unit='s')
df=df[['time', 'open', 'high', 'low', 'close']]
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close']
df=df.set_index('Datetime')

In [233]:
## QUICKSTART

In [351]:
## Functions to get indicators

def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(span=n, min_periods=n).mean() / loss.abs().ewm(span=n, min_periods=n).mean()
    return 100 - 100 / (1 + rs)

def MA_RSI(array, n):
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = (gain.ewm(span=n, min_periods=n).mean() / loss.abs().ewm(span=n, min_periods=n).mean()).rolling(10).mean()
    return 100 - 100 / (1 + rs)

def get_fdi(data, n):
    close = data['close']
    highest = np.max(data['high'])
    lowest = np.min(data['low'])
    length = 0
    pdiff = 0
    
    for period in range(n) :
        if (highest - lowest) > 0:
            diff = (close[period] - lowest) / (highest - lowest)
            if period > 0 :
                length += sqrt((diff - pdiff) ** 2 + 1 / (n ** 2))
            pdiff = diff
    
    if length > 0 :
        fdi = 1 + np.log(2 * length) / np.log(2 * n)
    else :
        fdi = 0
        
    return fdi


In [352]:
class SignalStrategy(Strategy):

    ## Params to Optimize on

    rsi_period = 14
    ma1_period = 21
    ma2_period = 50
    ma3_period = 100
    fdi_period = 14

    def init(self):

        self.ma21 = self.I(SMA, self.data.Close, self.ma1_period)
        self.ma50 = self.I(SMA, self.data.Close, self.ma2_period)
        self.ma100 = self.I(SMA, self.data.Close, self.ma3_period)

        self.rsi = self.I(RSI, self.data.Close, self.rsi_period)
        self.ma_rsi = self.I(MA_RSI, self.data.Close, self.rsi_period)

    def next(self):

        price = self.data.Close[-1]

        if (not self.position and price > self.ma21[-1] and price > self.ma50[-1] and price > self.ma100[-1] and self.rsi[-1] > 50):
            self.buy()
        else:
            self.sell()
            

In [353]:
bt = Backtest(df, SignalStrategy, cash=10_000)

In [354]:
stats = bt.run()

print(stats['_trades'].to_string())

     Size  EntryBar  ExitBar  EntryPrice  ExitPrice        PnL  ReturnPct           EntryTime            ExitTime         Duration
0      -2      6452     6492     1.06611    1.06866   -0.00510  -0.002392 2023-05-31 11:55:00 2023-05-31 15:15:00  0 days 03:20:00
1      -5      6451     6492     1.06636    1.06866   -0.01150  -0.002157 2023-05-31 11:50:00 2023-05-31 15:15:00  0 days 03:25:00
2      -2      6450     6492     1.06646    1.06866   -0.00440  -0.002063 2023-05-31 11:45:00 2023-05-31 15:15:00  0 days 03:30:00
3      -1      6449     6492     1.06652    1.06866   -0.00214  -0.002007 2023-05-31 11:40:00 2023-05-31 15:15:00  0 days 03:35:00
4      -2      6448     6492     1.06697    1.06866   -0.00338  -0.001584 2023-05-31 11:35:00 2023-05-31 15:15:00  0 days 03:40:00
5      -6      6443     6492     1.06696    1.06866   -0.01020  -0.001593 2023-05-31 11:10:00 2023-05-31 15:15:00  0 days 04:05:00
6      -3      6442     6492     1.06714    1.06866   -0.00456  -0.001424 2023-05-3

In [355]:
print(stats)

Start                     2023-05-01 00:05:00
End                       2023-05-31 15:15:00
Duration                     30 days 15:10:00
Exposure Time [%]                   98.444479
Equity Final [$]                  10298.13277
Equity Peak [$]                   10322.54373
Return [%]                           2.981328
Buy & Hold Return [%]               -3.057632
Return (Ann.) [%]                   34.319276
Volatility (Ann.) [%]                 7.18434
Sharpe Ratio                         4.776956
Sortino Ratio                       12.660366
Calmar Ratio                        25.619983
Max. Drawdown [%]                   -1.339551
Avg. Drawdown [%]                   -0.134866
Max. Drawdown Duration        7 days 22:30:00
Avg. Drawdown Duration        0 days 09:01:00
# Trades                                  145
Win Rate [%]                        91.034483
Best Trade [%]                       2.902053
Worst Trade [%]                     -0.239187
Avg. Trade [%]                    

In [272]:
bt.plot()

C:\Users\Utilisateur\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Utilisateur\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p11838', ...)

## Optimization

In [ ]:
## Pick which stat to opti, add constraints and choose param

stat_opti = bt.optimize(
    rsi_period = range(10, 25, 1),
    ma1_period = range(10, 25, 1),
    ma2_period = range(40, 60, 1),
    ma3_period = range(70, 150, 1),
    fdi_period = range(5, 20, 1),
    maximize ='Profit Factor',
    return_heatmap= True
)

In [ ]:
hm = heatmap.groupby(['upper_bound', 'lower_bound']).mean().unstack()

sns.heatmap(hm, cmap = 'viridis')

plt.show()

In [ ]:
## If more than 2 param to opti

plot_heatmaps(heatmap, agg='mean')